## Import Libraries

In [48]:
import os
import glob
import pandas as pd
from tqdm import tqdm

In [49]:
DISORDERS = ['agoraphobia', 'panic', 'phobia', 'selectivemutism', 'socialanxiety', 'anxiety']

In [50]:
# *** Reddit ***
reddit_df = pd.DataFrame()

for disorder_file_name in tqdm(glob.glob("../Data/Anxiety_Detection_Data/Reddit/*")):
    disorder_df_temp = pd.read_excel(disorder_file_name)
    disorder_df = pd.DataFrame({
        "text": disorder_df_temp['Translated_Text'],
        "labels": os.path.basename(disorder_file_name).split('.')[0],
        "source": "Reddit"
    })

    if len(reddit_df) != 0:
        reddit_df = pd.concat([reddit_df, disorder_df], axis=0).reset_index(drop=True)

    else:
        reddit_df = disorder_df.copy()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.51it/s]


In [51]:
# *** Eksi ***
eksi_df = pd.DataFrame()

for disorder_file_name in tqdm(glob.glob("../Data/Anxiety_Detection_Data/Eksi/*")):
    disorder_df_temp = pd.read_excel(disorder_file_name)
    disorder_df = pd.DataFrame({
        "text": disorder_df_temp['Text'],
        "labels": os.path.basename(disorder_file_name).split('.')[0],
        "source": "Eksi"
    })

    if len(eksi_df) != 0:
        eksi_df = pd.concat([eksi_df, disorder_df], axis=0).reset_index(drop=True)

    else:
        eksi_df = disorder_df.copy()

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 20.32it/s]


In [52]:
# *** Youtube ***
youtube_df = pd.read_excel("../Data/Anxiety_Detection_Data/Youtube/anxiety_youtube_data.xlsx")
youtube_df = youtube_df.rename(columns = {"Text": "text", "Disorder": "labels"})
youtube_df['source'] = "Youtube"

## Concat

In [53]:
total_df = pd.concat([reddit_df, eksi_df, youtube_df], axis = 0).reset_index(drop=True)
total_df

,text,labels,source
0,"Sıkışmış hissetmek (yerine tekrar giriş yok, y...",agoraphobia,Reddit
1,Yakın zamanda başka bir şehre taşındım ve nele...,agoraphobia,Reddit
2,"Panik atak geçirmenin eşiğindeydim, sadece bun...",agoraphobia,Reddit
3,Geçen hafta maruz kalma terapimi yaparken soka...,agoraphobia,Reddit
4,Bazen özgüven ve benlik imajıyla çok mücadele ...,agoraphobia,Reddit
...,...,...,...
19139,Benim kızım 7 yaşlnda anaokuluna göderdim öğre...,selectivemutism,Youtube
19140,Merhaba oğlum da anasınıfına gidiyor kimseyle ...,selectivemutism,Youtube
19141,benim de 5 buçuk yaşında oğlum aile bireyleri ...,selectivemutism,Youtube
19142,merhaba benim oğlumda okulda çocuklarla sohbet...,selectivemutism,Youtube


In [54]:
# total_df.to_excel("../Data/Anxiety_Detection_Data/total_df.xlsx", index=False)
# print("Done")

Done


In [44]:
total_df['labels'].value_counts()

labels
socialanxiety      8752
anxiety            3422
agoraphobia        2478
selectivemutism    1845
panic              1394
phobia             1253
Name: count, dtype: int64

## Convert them to Multi Label

In [45]:
total_df[['text', 'labels']]

,text,labels
0,"Sıkışmış hissetmek (yerine tekrar giriş yok, y...",agoraphobia
1,Yakın zamanda başka bir şehre taşındım ve nele...,agoraphobia
2,"Panik atak geçirmenin eşiğindeydim, sadece bun...",agoraphobia
3,Geçen hafta maruz kalma terapimi yaparken soka...,agoraphobia
4,Bazen özgüven ve benlik imajıyla çok mücadele ...,agoraphobia
...,...,...
19139,Benim kızım 7 yaşlnda anaokuluna göderdim öğre...,selectivemutism
19140,Merhaba oğlum da anasınıfına gidiyor kimseyle ...,selectivemutism
19141,benim de 5 buçuk yaşında oğlum aile bireyleri ...,selectivemutism
19142,merhaba benim oğlumda okulda çocuklarla sohbet...,selectivemutism


In [46]:
one_hot_encoded = pd.get_dummies(total_df['labels'])

total_df = pd.concat([total_df['text'], total_df['source'], one_hot_encoded], axis=1)

total_df.iloc[:, 2:] = total_df.iloc[:, 2:].astype(int)

total_df['all_labels'] = total_df.apply(lambda row: [label for label in one_hot_encoded.columns if row[label] == 1], axis=1)

In [47]:
total_df

,text,source,agoraphobia,anxiety,panic,phobia,selectivemutism,socialanxiety,all_labels
0,"Sıkışmış hissetmek (yerine tekrar giriş yok, y...",Reddit,1,0,0,0,0,0,[agoraphobia]
1,Yakın zamanda başka bir şehre taşındım ve nele...,Reddit,1,0,0,0,0,0,[agoraphobia]
2,"Panik atak geçirmenin eşiğindeydim, sadece bun...",Reddit,1,0,0,0,0,0,[agoraphobia]
3,Geçen hafta maruz kalma terapimi yaparken soka...,Reddit,1,0,0,0,0,0,[agoraphobia]
4,Bazen özgüven ve benlik imajıyla çok mücadele ...,Reddit,1,0,0,0,0,0,[agoraphobia]
...,...,...,...,...,...,...,...,...,...
19139,Benim kızım 7 yaşlnda anaokuluna göderdim öğre...,Youtube,0,0,0,0,1,0,[selectivemutism]
19140,Merhaba oğlum da anasınıfına gidiyor kimseyle ...,Youtube,0,0,0,0,1,0,[selectivemutism]
19141,benim de 5 buçuk yaşında oğlum aile bireyleri ...,Youtube,0,0,0,0,1,0,[selectivemutism]
19142,merhaba benim oğlumda okulda çocuklarla sohbet...,Youtube,0,0,0,0,1,0,[selectivemutism]
